This step calculates the integrals of motion for each star. Namely Energy, and angular momentum

In [1]:
#Load in relevant libraries
import math

import numpy as np

from astropy import units as u
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord, ICRS, Galactocentric, Distance, LSR
from astropy.coordinates import FK5
from astropy.coordinates import CartesianRepresentation, CartesianDifferential
from astropy.table import Table, vstack, hstack
from astropy.table import Column
from astropy.io import ascii

import gala.potential as gp
import gala.dynamics as gd
from gala.units import galactic

from scipy import stats

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler, normalize


import time

from matplotlib import pyplot

import pandas as pd

In [2]:
#Load in the subset that was created in Step 1
t = Table.read('Step_1_Output.fits')

print("Total Number of Stars in Dataset: " + str(len(t)))

Total Number of Stars in Dataset: 6789


In [3]:
#Using the galactocentric positions and velocities provided in 
# the data set the actions are calculated

x = t['x_val']
y = t['y_val']
z = t['z_val']

U = t['U']
V = t['V']
W = t['W']

#In order to calculate the actions, you need to specifiy a graviational potential. 
# In this case we use a combination of potentials in the same manner as Helmi et al 2017. 
#  Using the MilkyWayPotential class models the buldge, disk and halo in combination. 
pot = gp.MilkyWayPotential(units=galactic)

#We use the Gala function PhaseSpacePosition to save the 6-D phase space coordinates, which
# are taken from the above vectors,
w0 = gd.PhaseSpacePosition(pos=[x, y, z]*u.pc,
                           vel=[U, V, W]*u.km/u.s)

#The orbit of the star is then calculated using the following command for 1000 steps. 
# Each step is one megayear.
orbit = gp.Hamiltonian(pot).integrate_orbit(w0, dt=1., n_steps=1000)

#The energies of each stars orbit is calculated
E = orbit.energy().to(u.km*u.km/(u.s*u.s))
Energy = Column(E[0], name = 'Energy')


#Z-coomponent of angular momentum is calculated
Lz = orbit.angular_momentum()[2].to(u.km*u.kpc/u.s)
L_z = Column(Lz[0], name = 'Lz')

#The x and y components of the angular momentumes are caluclated
Lx = orbit.angular_momentum()[0].to(u.km*u.kpc/u.s)
Ly = orbit.angular_momentum()[1].to(u.km*u.kpc/u.s)

#We calculcualte the perpedicular component of the angular momentum, by applying
# pythagoras theorem to the x and y component
Lp = np.sqrt(Lx[0]*Lx[0]+Ly[0]*Ly[0])
L_perp = Column(Lp, name = 'L_perp')

#The calculated actions are then added as columns to the subset table
t.add_column(Energy)

#Gala's angular momentum values in the z-direction are negatives to Koppelman et al. 2018
# to get them to correspond we mulitply by minus one. 
L_z = -L_z
t.add_column(L_z)
t.add_column(L_perp)

print("Total Number of Stars in Dataset: " + str(len(t)))

Total Number of Stars in Dataset: 6789


In [4]:
#The subset is saved as a fits datafile ready to be datamined
t.write('Step_2_Output.fits',overwrite=True)